# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [11]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [12]:
!cat talks.tsv











Restricted power domination	Talk	talk-10	Special Session on The Inverse Eigenvalue Problem of a Graph and Zero Forcing, 52nd Southeastern International Conference on Combinatorics, Graph Theory, and Computing	2021-03-09	Boca Raton, FL		

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [13]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Strongly Eventually Nonnegative Matrices,Talk,talk-1,"Special Session on Matrices and Graphs, AMS Ce...",2010-04-11,"Saint Paul, MN",NaN,NaN
1,Potentially eventually positive and potentiall...,Talk,talk-2,Young Researchers Minisymposium on Combinatori...,2011-08-23,"Braunschweig, Germany",NaN,NaN
2,Positive semidefinite maximum nullity is equal...,Talk,talk-3,"Special Session on Matrices and Graphs, AMS Ce...",2011-10-14,"Lincoln, NE",NaN,NaN
3,Sign Patterns That Allow Strong Eventual Nonne...,Talk,talk-4,"Minisymposium on Matrices and Graphs, 2012 SIA...",2012-07-12,"Minneapolis, MN",NaN,NaN
4,Sign patterns that require eventual exponentia...,Talk,talk-5,Special Session on Generalizations of Nonnegat...,2013-04-27,"Ames, IA",NaN,NaN
5,Sign patterns that require eventual exponentia...,Talk,talk-6,"Minisymposium on Sign Pattern Matrices, 18th I...",2013-06-06,"Providence, RI",NaN,NaN
6,Upper triangular sign patterns that require ev...,Talk,talk-7,"Minisymposium on Combinatorial Matrix Theory, ...",2016-07-15,"Leuven, Belgium",NaN,NaN
7,Upper triangular sign patterns that require ev...,Talk,talk-8,Special Session on Combinatorial Matrix Theory...,2016-10-30,"Minneapolis, MN",NaN,NaN
8,Zero forcing and other games on graphs,Talk,talk-9,Mathematics & Statistics Department Colloquium...,2018-04-17,"Northfield, MN",NaN,NaN
9,Restricted power domination,Talk,talk-10,Special Session on The Inverse Eigenvalue Prob...,2021-03-09,"Boca Raton, FL",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [14]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [15]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [16]:
!ls ../_talks

2010-04-11-talk-1.md  2013-04-27-talk-5.md  2018-04-17-talk-9.md
2011-08-23-talk-2.md  2013-06-06-talk-6.md  2021-03-09-talk-10.md
2011-10-14-talk-3.md  2016-07-15-talk-7.md  talkmap.ipynb
2012-07-12-talk-4.md  2016-10-30-talk-8.md


In [18]:
!cat ../_talks/2021-03-09-talk-10.md

---
title: "Restricted power domination"
collection: talks
type: "Talk"
permalink: /talks/2021-03-09-talk-10
venue: "Special Session on The Inverse Eigenvalue Problem of a Graph and Zero Forcing, 52nd Southeastern International Conference on Combinatorics, Graph Theory, and Computing"
date: 2021-03-09
location: "Boca Raton, FL"
---
